### Key Instance Detection for Conformers

KID for conformers

In [ ]:
import logging
import warnings
warnings.filterwarnings("ignore")
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("lightning").setLevel(logging.ERROR)

import time
import torch
import pickle
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Preprocessing
from milearn.preprocessing import BagMinMaxScaler

# Network hparams
from milearn.network.module.hopt import DEFAULT_PARAM_GRID

# MIL wrappers
from milearn.network.regressor import BagWrapperMLPNetworkRegressor, InstanceWrapperMLPNetworkRegressor

# MIL networks
from milearn.network.regressor import (InstanceNetworkRegressor,
                                       BagNetworkRegressor,
                                       AdditiveAttentionNetworkRegressor,
                                       SelfAttentionNetworkRegressor,
                                       HopfieldAttentionNetworkRegressor,
                                       DynamicPoolingNetworkRegressor)

# Utils
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split

# 3D descriptors
from qsarmil.descriptor.rdkit import (RDKitGEOM, 
                                      RDKitAUTOCORR, 
                                      RDKitRDF, 
                                      RDKitMORSE, 
                                      RDKitWHIM, 
                                      RDKitGETAWAY)
from molfeat.calc import Pharmacophore3D, USRDescriptors, ElectroShapeDescriptors
from qsarmil.descriptor.wrapper import DescriptorWrapper

# Data
from huggingface_hub import hf_hub_download

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol
import numpy as np
from IPython.display import display, HTML

def visualize_conformers_grid(mol, weights, key_conformers, top_n=5, 
                              style="stick", n_cols=3, width=250, height=250,
                              show_all=False, sort_by_weight=True):

    num_confs = mol.GetNumConformers()
    if num_confs != len(weights):
        raise ValueError("Number of weights must equal number of conformers")

    # top-N predicted indices
    top_indices = set(np.argsort(weights)[-top_n:][::-1])
    key_conformers = set(key_conformers)

    if show_all:
        conf_indices = list(range(num_confs))
    else:
        conf_indices = sorted(key_conformers.union(top_indices))

    # sort conformers by weight if requested
    if sort_by_weight:
        conf_indices = sorted(conf_indices, key=lambda i: weights[i], reverse=True)

    viewers_html = []
    for i in conf_indices:
        conf = mol.GetConformer(int(i) + 1)
        block = Chem.MolToMolBlock(mol, confId=conf.GetId())

        color = "0xAAAAAA"  # default grey
        label = f"Conf {i} (w={weights[i]:.2f})"
        if i in key_conformers:
            color = "0xFF0000"  # red
            label += " [TRUE]"
        elif i in top_indices:
            color = "0x0000FF"  # blue
            label += " [PRED]"

        viewer = py3Dmol.view(width=width, height=height)
        viewer.addModel(block, "sdf")
        viewer.setStyle({style: {"color": color}})
        viewer.zoomTo()
        
        html = viewer._make_html()
        viewers_html.append(f"<div style='display:inline-block; text-align:center;'>{html}<br>{label}</div>")

    # arrange into grid
    rows = []
    for i in range(0, len(viewers_html), n_cols):
        row_html = "".join(viewers_html[i:i+n_cols])
        rows.append(f"<div style='margin-bottom:20px'>{row_html}</div>")

    # add legend
    legend_html = """
    <div style='margin:10px 0;'>
      <b>Legend:</b> 
      <span style='color:red;'>[TRUE]=Ground truth</span> | 
      <span style='color:blue;'>[PRED]=Top predicted</span> | 
      <span style='color:gray;'>Others</span>
    </div>
    """

    display(HTML(legend_html + "".join(rows)))

In [ ]:
from math import comb

def kid_accuracy(true_key_indices, predicted_weights, top_n=1):
    assert len(predicted_weights) == len(true_key_indices), "Mismatched input lengths."

    hits = 0
    total = len(predicted_weights)

    for bag_weights, key_indices in zip(predicted_weights, true_key_indices):

        top_n_indices = sorted(range(len(bag_weights)), key=lambda i: bag_weights[i], reverse=True)[:top_n]

        if any(idx in top_n_indices for idx in key_indices):
            hits += 1

    return hits / total if total > 0 else 0.0

def expected_kid_accuracy(true_key_indices, bag_sizes, top_n=1):

    assert len(true_key_indices) == len(bag_sizes), "Mismatched input lengths."
    expected_hits = 0

    for key_indices, B in zip(true_key_indices, bag_sizes):
        K = len(key_indices)
        N = min(top_n, B)  # top_n can't exceed bag size

        if K == 0 or B == 0:
            continue  # skip invalid bags

        if B - K < N:
            hit_prob = 1.0  # guaranteed to pick a key instance
        else:
            hit_prob = 1 - (comb(B - K, N) / comb(B, N))

        expected_hits += hit_prob

    return expected_hits / len(true_key_indices) if true_key_indices else 0.0

### 1. Load data

In [ ]:
REPO_ID = "KagakuData/notebooks"

pkl_path = hf_hub_download(REPO_ID, filename="conformer/train_conf.pkl", repo_type="dataset")
with open(pkl_path, "rb") as f:
    data_train = pickle.load(f)

pkl_path = hf_hub_download(REPO_ID, filename="conformer/test_conf.pkl", repo_type="dataset")
with open(pkl_path, "rb") as f:
    data_test = pickle.load(f)

In [ ]:
# molecules
mol_train = [i[1] for i in data_train]
mol_test = [i[1] for i in data_test]

# property
y_train = [i[3] for i in data_train]
y_test = [i[3] for i in data_test]

# key instances
key_train = [i[2] for i in data_train]
key_test = [i[2] for i in data_test]

### 2. Build model

In [ ]:
desc_calc = DescriptorWrapper(Pharmacophore3D(factory='pmapper'))

# 1. Calc descriptors
x_train = desc_calc.transform(mol_train)
x_test = desc_calc.transform(mol_test)

# 2. Scale descriptors
scaler = BagMinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
model = DynamicPoolingNetworkRegressor()

model.hopt(x_train_scaled, y_train, param_grid=DEFAULT_PARAM_GRID, verbose=True)
model.fit(x_train_scaled, y_train)

In [ ]:
y_pred = model.predict(x_test_scaled)
w_pred = model.get_instance_weights(x_test_scaled)
w_pred = [w.flatten() for w in w_pred]

In [ ]:
top_n = 1

print(f"All molecules: {len(y_test)}")
print(f"Prediction accuracy: {r2_score(y_test, y_pred):.2f}")
print(f"KID prediction accuracy: {kid_accuracy(key_test, w_pred, top_n=top_n):.2f}")
print(f"KID baseline accuracy: {expected_kid_accuracy(key_test, [len(i) for i in w_pred], top_n=top_n):.2f}")

idx_7 = []
for n, y in enumerate(y_test):
    if y == 7:
        idx_7.append(n)
key_test_7 = [key_test[i] for i in idx_7]
w_pred_7 = [w_pred[i] for i in idx_7]

print(f"\nActive molecules: {len(idx_7)}")
print(f"KID prediction accuracy: {kid_accuracy(key_test_7, w_pred_7, top_n=top_n):.2f}")
print(f"KID baseline accuracy: {expected_kid_accuracy(key_test_7, [len(i) for i in w_pred_7], top_n=top_n):.2f}")

In [ ]:
idx_7[:15]

In [ ]:
N = 45

visualize_conformers_grid(mol_test[N], w_pred[N], key_test[N], top_n=5, 
                          style="stick", n_cols=4, width=250, height=250, show_all=False, sort_by_weight=True)

### 3. Mini-benchmark

In [ ]:
desc_list = [
             ("MolFeatPmapper", DescriptorWrapper(Pharmacophore3D(factory='pmapper'))), # can be long
            ]

In [ ]:
regressor_list = [

        # attention mil networks
        ("AdditiveAttentionNetworkRegressor", AdditiveAttentionNetworkRegressor()),
        ("SelfAttentionNetworkRegressor", SelfAttentionNetworkRegressor()),
        ("HopfieldAttentionNetworkRegressor", HopfieldAttentionNetworkRegressor()),

        # other mil networks
        ("DynamicPoolingNetworkRegressor", DynamicPoolingNetworkRegressor()),
    ]

In [ ]:
n = 0
total_n = len(desc_list) * len(regressor_list)

res_df = pd.DataFrame()
for desc_name, desc_calc in desc_list:

    # 1. Calc descriptors
    x_train = desc_calc.transform(mol_train)
    x_test = desc_calc.transform(mol_test)

    # 2. Scale descriptors
    scaler = BagMinMaxScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    for method_name, model in regressor_list:
        
        # 3. Train model
        # model.hopt(x_train_scaled, y_train, param_grid=DEFAULT_PARAM_GRID, verbose=False)
        model.fit(x_train_scaled, y_train)
        
        # 4. Get predictions
        y_pred = model.predict(x_test_scaled)
        res_df.loc[f"{desc_name}|{method_name}", "ACC"] = r2_score(y_test, y_pred)

        # 5. Key instance detection
        w_pred = model.get_instance_weights(x_test_scaled)
        w_pred = [w.flatten() for w in w_pred]

        # calc kid accuracy
        for top_n in [1, 2, 3]:
            res_df.loc[f"{desc_name}|{method_name}", f"TOP-{top_n}"] = kid_accuracy(key_test, w_pred, top_n=top_n)
        
        # logging
        n += 1
        print(f"{n}/{total_n} {desc_name}|{method_name}", end="\r")

In [ ]:
res_df.round(2)